In [ ]:
# This magic makes plots appear in the browser
%matplotlib inline
import matplotlib.pyplot as plt

# Load Firedrake on Colab
try:
    import firedrake
except ImportError:
    !wget "https://github.com/thwaitesproject/tutorials/releases/latest/download/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

try: 
    import thwaites
except:
    !pip install git+https://github.com/thwaitesproject/thwaites
    import thwaites



In [ ]:
from thwaites import *
from thwaites.utility import FrazilRisingVelocity
from firedrake.petsc import PETSc
from firedrake import FacetNormal
import numpy as np

# Ice shelf basal crevasses 
Investigations by Automatic Underwater Vehicles (AUV) equipped with upward looking sonar systems have shown that the landscape of basal features is far from flat. Crevasses, channels and curious step features, referred to as basal terraces, are common features found on a number of different ice shelves.

![basal_terraces.jpg](basal_terraces.jpg)

<h2>Frazil Ice Dynamics**</h2>

**If you are interested!

We have implemented a frazil ice model based on the work of Jordan et al, 2014. Frazil ice contributes to the density, $\rho$ of the water parcel according to

\begin{equation}
\rho = \rho_0(1-C) (1 + \rho') +  \rho_i C,
\end{equation}
where $C$ is the fraction of frazil ice, $\rho_0$ is the bulk density of sea water, $\rho_i$ is the density of ice and the density perturbation, $\rho'$, is given by
\begin{equation}
\rho' = -\alpha_T(T - T_{ref}) + \beta_S (S - S_{ref}).
\end{equation}
$S_{ref}$ and $T_{ref}$ are the expansion coefficients from the linear equation of state. Temperature, $T$, is an active tracer governed by an advection diffusion equation
\begin{equation}
\dfrac{\partial T}{\partial t} + \textbf{u} \cdot \nabla T   =  \nabla \cdot \kappa_T \nabla T + \left( T_c - T - \frac{L}{c_p}\right) w_c.
\end{equation}
The source term on the right hand side accounts for the latent heat release when frazil ice crystals form. $w_c$ is the melt rate for frazil ice crystals ($w_c <$  0 is freezing). $T_c$ is the temperature at the interface between the frazil ice and the ocean. $L$ is the latent heat of fusion of water and $c_p$ is the specific heat capacity of water.
Similarly salinity, $S$, evolves as
\begin{equation}
\dfrac{\partial S}{\partial t} + \textbf{u} \cdot \nabla S  =  \nabla \cdot \kappa_S \nabla S - S w_c.
\end{equation}
The frazil ice is also modelled by an advection diffusion equation with an additional advection term that causes the ice to rise, equivalent to a negative `sinking' velocity used in sediment models given by
\begin{equation}
\dfrac{\partial C}{\partial t} + \textbf{u} \cdot \nabla C + w_i \dfrac{\partial C}{\partial z} =  \nabla \cdot \kappa_C \nabla C - w_c.
\end{equation}
The rising velocity $w_i$ is given by
\begin{equation}
w_i^2 =  \frac{4 R g r \epsilon }{C_d},
\end{equation}
where $g$ is gravity, $r$ is the radius of the crystal, $\epsilon$ is the aspect ratio of the crystal and $R$ is the specific gravity given by
\begin{equation}
R = \frac{\rho_i - \rho_0 }{\rho_0}.
\end{equation}
The drag coefficient $C_d$ is 
\begin{equation}
\text{log}_{10}(C_d) =  1.386 - 0.892 \: \text{log}_{10}(Re) + 0.111 (\text{log}_{10}(Re))^2,
\end{equation}
where the disk Reynolds number, $Re$, is given by
\begin{equation}
Re =  \frac{w_i 2 r }{\nu},
\end{equation}
where $\nu$ is the viscosity of sea water. These equations are solved iteratively to give a unique rising velocity for a given radius of frazil ice crystal.

The freezing rate due to frazil ice is taken to be the flux of frazil ice through the top boundary given by
\begin{equation}
\dfrac{\partial \eta}{\partial t} =  w_b C_b,
\end{equation}
where $w_b$ is the velocity of the frazil ice at the top boundary and $C_b$ is the concentration at the top boundary. 
Melt and freezing rates of frazil ice are calculated based on conservation of heat and salt at the frazil interface and that the temperature at the frazil interface is at the freezing point. 
The conservation of heat is given by
\begin{equation}
(1-C)\gamma_T^c (T-T_c) \dfrac{2C}{r} = \frac{L}{c_p} w_c,
\end{equation}
where $\gamma_T^c$ is the thermal exchange coefficient and is given by
\begin{equation}
\gamma_T^c = \dfrac{Nu \kappa_T}{\epsilon r}.
\end{equation}
$Nu$ is the Nusselt number taken here to be 1 and $\kappa_T$ is the molecular thermal diffusivity of sea water. 

The conservation of salt is given by
\begin{equation}
(1-C)\gamma_S^c (S-S_c) \dfrac{2C}{r} = S_c w_c,
\end{equation}
where $\gamma_S^c$ is the equivalent salt exchange coefficient, given by
\begin{equation}
\gamma_S^c = \dfrac{Nu \kappa_S}{\epsilon r},
\end{equation}
and $\kappa_S$ is the mass diffusivity of sea water. 

The freezing point is given by
\begin{equation}
T_c  =  a S_c + b + c p_c,
\end{equation}
where $a$, $b$ and $c$ are constants obtained from the linearisation. Importantly the freezing point depends on pressure so frazil ice is more likely to form near the top of crevasses.


<h2>Model Setup</h2>

This a dynamic setup that is a good demonstration of the ability to resolve complex overturning flow robustly on unstructured meshes. Jordan et al. 2014 carried out an investigation of an idealised 2D cavity based on the Jutulgryta rift in the Fimbulisen ice shelf. The study site was chosen because observations of temperature and salinity were available. An approximate freezing rates of 1 m/a was estimated by due to 2 m of ice build up when retrieving the instruments two years later. 

The setup is intended to match the baseline case from Jordan et al. 2014. The rift is 340 m wide and 260 m deep. The top of the rift is 40 m below the surface. Beneath the crevasse the ocean cavity is 100 m thick with a flat ice base 300 m deep and a flat seabed 400 m deep. The total horizontal extent of the domain is 5 km. The mesh was generated using Gmsh. The mesh is made up of triangles with variable resolution. For this tutorial we have coarsened the resolution to 15 m within the crevasse (Jordan et al. 2014 used 5 m) and becomes coarser outside the crevasse to 25 m. No normal flow is imposed on the top and bottom of the domain. There is an inflow on the left boundary of 0.025 m/s and on the right boundary is open. Flux boundary conditions for temperature and salinity are imposed along the ice base (including the verticals walls inside the crevasse). The fluxes are calculated using the ‘three-equation’ melt rate parameterisation as described in the previous example. 



In [ ]:
try: 
    # create mesh
    mesh = Mesh("./Crevasse_coarse.msh")
except:
    # load mesh
    !wget https://raw.githubusercontent.com/thwaitesproject/tutorials/main/Crevasse_coarse.msh
    mesh = Mesh("./Crevasse_coarse.msh")



PETSc.Sys.Print("Mesh dimension ", mesh.geometric_dimension())

# shift z = 0 to surface of ocean. N.b z = 0 is outside domain.
PETSc.Sys.Print("Length of lhs", assemble(Constant(1.0)*ds(3, domain=mesh)))

PETSc.Sys.Print("Length of rhs", assemble(Constant(1.0)*ds(2, domain=mesh)))

PETSc.Sys.Print("Length of bottom", assemble(Constant(1.0)*ds(1, domain=mesh)))

PETSc.Sys.Print("Length of top", assemble(Constant(1.0)*ds(4, domain=mesh)))




print("You have Comm WORLD size = ", mesh.comm.size)
print("You have Comm WORLD rank = ", mesh.comm.rank)

y, z = SpatialCoordinate(mesh)
water_depth = 400

In [ ]:
# Set up function spaces
V = VectorFunctionSpace(mesh, "DG", 1)  # velocity space
W = FunctionSpace(mesh, "CG", 2)  # pressure space
M = MixedFunctionSpace([V, W])

# u velocity function space.
U = FunctionSpace(mesh, "DG", 1)

Q = FunctionSpace(mesh, "DG", 1)  # melt function space
K = FunctionSpace(mesh, "DG", 1)    # temperature space
S = FunctionSpace(mesh, "DG", 1)    # salinity space

In [ ]:
# Set up functions
m = Function(M)
v_, p_ = m.split()  # function: y component of velocity, pressure
v, p = split(m)  # expression: y component of velocity, pressure
v_._name = "v_velocity"
p_._name = "perturbation pressure"

rho = Function(K, name="density")
temp = Function(K, name="temperature")
sal = Function(S, name="salinity")
melt = Function(Q, name="melt rate")
Q_mixed = Function(Q, name="ocean heat flux")
Q_ice = Function(Q, name="ice heat flux")
Q_latent = Function(Q, name="latent heat")
Q_s = Function(Q, name="ocean salt flux")
Tb = Function(Q, name="boundary freezing temperature")
Sb = Function(Q, name="boundary salinity")
full_pressure = Function(M.sub(1), name="full pressure")

frazil = Function(Q, name="frazil ice concentration") # should this really be P0dg to prevent negative frazil ice?
frazil_flux = Function(Q, name="frazil ice flux") 

<h2>Initial conditions</h2>

Temperature and salinity are initialised at the baseline values from Jordan et al. 2014: -1.965 $^\circ$C and 34.34 respectively. This puts the depth dependent freezing point within the crevasse and ensures that frazil ice forms within the cavity. These values are also used as Dirichlet boundary conditions on the left-hand inflow boundary. All boundary conditions are imposed weakly.  The velocity and pressure are discretised using the P1DG-P2 finite element pair and the tracers are discretised using P1DG and a vertex based limiter. We use a constant viscosity and diffusivity of 1$\times10^{-3}$ m$^2$/s. Again, for this tutorial, we have increased the timestep to 180 s compared with 5 s for Jordan et al. 2014. 


In [ ]:
dump_file = "/data/2d_crevasse/17.02.22_3_eq_param_ufricHJ99_dt5.0_dtOutput3600.0_T864000.0_isotropicdx5to25m_open_iterative_0.025inflow_qice=0_400mdepth_frazil_sharpmesh_3changedensity_allsource_salsource_limfraz5e-9/dump_step_172800.h5"

DUMP = False
if DUMP:
    with DumbCheckpoint(dump_file, mode=FILE_UPDATE) as chk:
        # Checkpoint file open for reading and writing
        chk.load(v_, name="v_velocity")
        chk.load(p_, name="perturbation_pressure")
        chk.load(sal, name="salinity")
        chk.load(temp, name="temperature")
        chk.load(frazil, name="frazil ice concentration")

        T_200m_depth = -1.965

        S_200m_depth = 34.34
        #S_bottom = 34.8
        #salinity_gradient = (S_bottom - S_200m_depth) / -H2
        #S_surface = S_200m_depth - (salinity_gradient * (H2 - water_depth))  # projected linear slope to surface.

        T_restore = Constant(T_200m_depth)
        S_restore = Constant(S_200m_depth) #S_surface + (S_bottom - S_surface) * (z / -water_depth)


else:
    # Assign Initial conditions
    v_init = zero(mesh.geometric_dimension())
    v_.assign(v_init)

      # baseline T3
    T_200m_depth = -1.965


    #S_bottom = 34.8
    #salinity_gradient = (S_bottom - S_200m_depth) / -H2
    S_surface = 34.34 #S_200m_depth - (salinity_gradient * (H2 - water_depth))  # projected linear slope to surface.

    T_restore = Constant(T_200m_depth)
    S_restore = Constant(S_surface) #S_surface + (S_bottom - S_surface) * (z / -water_depth)

    temp_init = T_restore
    temp.interpolate(temp_init)

    sal_init = Constant(34.34)
    #sal_init = S_restore
    sal.interpolate(sal_init)
    
    frazil_init = Constant(5e-9) # initialise with a minimum frazil ice concentration
    frazil.interpolate(frazil_init)


In [ ]:
# Set up equations
mom_eq = MomentumEquation(M.sub(0), M.sub(0))
cty_eq = ContinuityEquation(M.sub(1), M.sub(1))
temp_eq = ScalarAdvectionDiffusionEquation(K, K)
sal_eq = ScalarAdvectionDiffusionEquation(S, S)
frazil_eq = FrazilAdvectionDiffusionEquation(Q,Q)

In [ ]:
# Terms for equation fields

# momentum source: the buoyancy term Boussinesq approx. From Jordan etal 14
T_ref = Constant(-2.0)
S_ref = Constant(34.5)
beta_temp = Constant(3.87E-5)
beta_sal = Constant(7.86E-4)
g = Constant(9.81)
rho0 = 1030.
rho_ice = 920.

rho_perb = -beta_temp*(temp - T_ref) + beta_sal * (sal - S_ref)  # Linear eos (already divided by rho0)
mom_source = as_vector((0., -g)) * (rho_perb - frazil * (1 + rho_perb) + frazil * (rho_ice / rho0))
rho.interpolate(rho0*((1-frazil) * (1 + rho_perb)) + frazil * rho_ice)
# coriolis frequency f-plane assumption at 75deg S. f = 2 omega sin (lat) = 2 * 7.2921E-5 * sin (-75 *2pi/360)
#f = Constant(-1.409E-4)

# Scalar source/sink terms at open boundary.
restoring_time = 86400.
absorption_factor = Constant(1.0/restoring_time)
sponge_fraction = 0.06  # fraction of domain where sponge
# Temperature source term


kappa = as_tensor([[1e-3, 0], [0, 1e-3]])

kappa_temp = kappa
kappa_sal = kappa
kappa_frazil = kappa
mu = kappa

# define time steps
dt = 180
T = 86400
output_dt = 3600
output_step = output_dt/dt



<h2>Adding Frazil Ice</h2>

In [ ]:
FRV = FrazilRisingVelocity(0.1)  # initial velocity guess needs to be >0
w_i = FRV.frazil_rising_velocity() # Picard iterations converge to value for w_i (which only depends on crystal size, here we assume r =7.5e-4m

frazil_mp = FrazilMeltParam(sal, temp, p, z, frazil)
temp_source = (frazil_mp.Tc - temp - frazil_mp.Lf/frazil_mp.c_p_m) * frazil_mp.wc
temp_absorption = 0 
sal_source = -sal *frazil_mp.wc
sal_absorption = 0 
frazil_source =  -frazil_mp.wc
frazil_absorption = 0

In [ ]:

# Equation fields
vp_coupling = [{'pressure': 1}, {'velocity': 0}]
vp_fields = {'viscosity': mu, 'source': mom_source} 
temp_fields = {'diffusivity': kappa_temp, 'velocity': v, 'source': temp_source, 'absorption coefficient': temp_absorption}
sal_fields = {'diffusivity': kappa_sal, 'velocity': v, 'source': sal_source, 'absorption coefficient': sal_absorption, }
frazil_fields = {'diffusivity': kappa_frazil, 'velocity': v, 'w_i': Constant(w_i), 'source': frazil_source, 'absorption coefficient': frazil_absorption}


In [ ]:
# Get expressions used in melt rate parameterisation
mp = ThreeEqMeltRateParam(sal, temp, p, z, velocity=pow(dot(v, v), 0.5), HJ99Gamma=True, ice_heat_flux=False)

In [ ]:
# assign values of these expressions to functions.
# so these alter the expression and give new value for functions.
Q_ice.interpolate(mp.Q_ice)
Q_mixed.interpolate(mp.Q_mixed)
Q_latent.interpolate(mp.Q_latent)
Q_s.interpolate(mp.S_flux_bc)
melt.interpolate(mp.wb)
Tb.interpolate(mp.Tb)
Sb.interpolate(mp.Sb)
full_pressure.interpolate(mp.P_full)
frazil_flux.interpolate(w_i*frazil)

<h2>Boundary Conditions</h2>

In [ ]:


# Boundary conditions
# top boundary: no normal flow, drag flowing over ice
# bottom boundary: no normal flow, drag flowing over bedrock
# open ocean(LHS): inflow
# open ocean (RHS): pressure to account for density differences

# WEAKLY Enforced BCs
n = FacetNormal(mesh)
Temperature_term = -beta_temp * ((T_restore-T_ref) * z)
Salinity_term = beta_sal * ((S_restore - S_ref) * z) # ((S_bottom - S_surface) * (pow(z, 2) / (-2.0*water_depth)) + (S_surface-S_ref) * z)
stress_open_boundary = -n*-g*(Temperature_term + Salinity_term)
no_normal_flow = 0.
ice_drag = 0.0097

vp_bcs = {4: {'un': no_normal_flow, 'drag': ice_drag}, 2: {'stress': stress_open_boundary}, 
        3: {'un': -0.025}, 1: {'un': no_normal_flow, 'drag': 2.5e-3}}


temp_bcs = {4: {'flux': -mp.T_flux_bc}, 3:{'q': T_restore}}

sal_bcs = {4: {'flux': -mp.S_flux_bc}, 3:{'q': S_restore}}

frazil_bcs = {}

<h2>Solver parameters</h2>

In [ ]:
# Solver parameters
mumps_solver_parameters = {
    'snes_monitor': None,
    'snes_type': 'ksponly',
    'ksp_type': 'preonly',
    'pc_type': 'lu',
    'pc_factor_mat_solver_type': 'mumps',
    'mat_type': 'aij',
    'snes_max_it': 100,
    'snes_rtol': 1e-8,
    'snes_atol': 1e-6,
}

pressure_projection_solver_parameters = {
        'snes_type': 'ksponly',
        'ksp_type': 'preonly',  # we solve the full schur complement exactly, so no need for outer krylov
        'mat_type': 'matfree',
        'pc_type': 'fieldsplit',
        'pc_fieldsplit_type': 'schur',
        'pc_fieldsplit_schur_fact_type': 'full',
        # velocity mass block:
        'fieldsplit_0': {
            'ksp_type': 'gmres',
            'pc_type': 'python',
            'pc_python_type': 'firedrake.AssembledPC',
            'ksp_converged_reason': None,
            'assembled_ksp_type': 'preonly',
            'assembled_pc_type': 'bjacobi',
            'assembled_sub_pc_type': 'ilu',
            },
        # schur system: explicitly assemble the schur system
        # this only works with pressureprojectionicard if the velocity block is just the mass matrix
        # and if the velocity is DG so that this mass matrix can be inverted explicitly
        'fieldsplit_1': {
            'ksp_type': 'preonly',
            'pc_type': 'python',
            'pc_python_type': 'thwaites.AssembledSchurPC',
            'schur_ksp_type': 'cg',
            'schur_ksp_max_it': 1000,
            'schur_ksp_rtol': 1e-7,
            'schur_ksp_atol': 1e-9,
            'schur_ksp_converged_reason': None,
            'schur_pc_type': 'gamg',
            'schur_pc_gamg_threshold': 0.01
            },
        }

vp_solver_parameters = pressure_projection_solver_parameters
u_solver_parameters = mumps_solver_parameters
temp_solver_parameters = mumps_solver_parameters
sal_solver_parameters = mumps_solver_parameters
frazil_solver_parameters = mumps_solver_parameters

<h2>Timesteppers</h2>

In [ ]:
# Set up time stepping routines

vp_timestepper = PressureProjectionTimeIntegrator([mom_eq, cty_eq], m, vp_fields, vp_coupling, dt, vp_bcs,
                                                          solver_parameters=vp_solver_parameters,
                                                          predictor_solver_parameters=u_solver_parameters,
                                                          picard_iterations=1)

# performs pseudo timestep to get good initial pressure
# this is to avoid inconsistencies in terms (viscosity and advection) that
# are meant to decouple from pressure projection, but won't if pressure is not initialised
# do this here, so we can see the initial pressure in pressure_0.pvtu
if not DUMP:
    # should not be done when picking up
    vp_timestepper.initialize_pressure()

temp_timestepper = DIRK33(temp_eq, temp, temp_fields, dt, temp_bcs, solver_parameters=temp_solver_parameters)
sal_timestepper = DIRK33(sal_eq, sal, sal_fields, dt, sal_bcs, solver_parameters=sal_solver_parameters)
frazil_timestepper = DIRK33(frazil_eq, frazil, frazil_fields, dt, frazil_bcs, solver_parameters=frazil_solver_parameters)

In [ ]:
# Output files for velocity, pressure, temperature and salinity
folder="crevasse/"
v_file = File(folder+"vw_velocity.pvd")
v_file.write(v_)

p_file = File(folder+"pressure.pvd")
p_file.write(p_)


t_file = File(folder+"temperature.pvd")
t_file.write(temp)

s_file = File(folder+"salinity.pvd")
s_file.write(sal)

rho_file = File(folder+"density.pvd")
rho_file.write(rho)

frazil_file = File(folder+"frazil.pvd")
frazil_file.write(frazil)

m_file = File(folder+"melt.pvd")
m_file.write(melt)

frazil_flux_file = File(folder+"frazil_flux.pvd")
frazil_flux_file.write(frazil_flux)

<h2>Timestepping</h2>

Let's go!

In [ ]:
# Add limiter for DG functions
limiter = VertexBasedP1DGLimiter(S)

# Begin time stepping
t = 0.0
step = 0

In [ ]:
while t < T - 0.5*dt:
    vp_timestepper.advance(t)
    temp_timestepper.advance(t)
    sal_timestepper.advance(t)
        #u_timestepper.advance(t)
    frazil_timestepper.advance(t)
    step += 1
    t += dt

    limiter.apply(sal)
    limiter.apply(temp)
    limiter.apply(frazil)
    frazil.interpolate(conditional(frazil < 5e-9, 5e-9, frazil))
    if step % output_step == 0:
          # dumb checkpoint for starting from last timestep reached
          with DumbCheckpoint(folder+"dump.h5", mode=FILE_UPDATE) as chk:
              # Checkpoint file open for reading and writing
              chk.store(v_, name="v_velocity")
              chk.store(p_, name="perturbation_pressure")
              chk.store(temp, name="temperature")
              chk.store(sal, name="salinity")
              chk.store(frazil, name="frazil ice concentration")
              # Update melt rate functions
          Q_ice.interpolate(mp.Q_ice)
          Q_mixed.interpolate(mp.Q_mixed)
          Q_latent.interpolate(mp.Q_latent)
          Q_s.interpolate(mp.S_flux_bc)
          melt.interpolate(mp.wb)
          Tb.interpolate(mp.Tb)
          Sb.interpolate(mp.Sb)
          full_pressure.interpolate(mp.P_full)
          frazil_flux.interpolate(w_i*frazil)
              # Update density for plotting
          rho.interpolate(rho0*((1-frazil)*(-beta_temp*(temp - T_ref) + beta_sal * (sal - S_ref)) + (rho_ice / rho0) * frazil))
          
          
          
          # Write out files
          v_file.write(v_)
          p_file.write(p_)
          #u_file.write(u)
          t_file.write(temp)
          s_file.write(sal)
          rho_file.write(rho)
          frazil_file.write(frazil)   
          # Write melt rate functions
          m_file.write(melt)
          frazil_flux_file.write(frazil_flux)
          time_str = str(step)

          PETSc.Sys.Print("t=", t)
          PETSc.Sys.Print("integrated melt =", assemble(melt * ds(4)))

    if step % (output_step * 24) == 0:
        with DumbCheckpoint(folder+"dump_step_{}.h5".format(step), mode=FILE_CREATE) as chk:
            # Checkpoint file open for reading and writing at regular interval
            chk.store(v_, name="v_velocity")
            chk.store(p_, name="perturbation_pressure")
            chk.store(temp, name="temperature")
            chk.store(sal, name="salinity")
            chk.store(frazil, name="frazil ice concentration")


In [ ]:
import pyvista as pv

temp_data = pv.read("crevasse/temperature_24.vtu")

boring_cmap = plt.cm.get_cmap("viridis", 25)
plotter = pv.Plotter(notebook=True)
plotter.add_mesh(temp_data, cmap=boring_cmap)
plotter.camera_position = "xy"
plotter.show(jupyter_backend="static", interactive=False)

<h1>Exercises</h1>

1) Try running the simulation for 1 day and plot how the frazil ice concentration, temperature field and velocity. What do you notice about how the flow evolves through time?  

2) Try adjusting the initial temperature conditions (and inflow boundary values) to vary the amount of frazil ice production. Can you switch off frazil formation completely? What temperature does this occur at? 

